In [ ]:
intial_date = date(2017, 1, 1)
final_date = date(2018, 1, 1)
sales_per_month_query = f'''
SELECT EXTRACT(MONTH FROM order_purchase_timestamp), SUM(PRICE)
FROM orders LEFT JOIN order_items ON orders.order_id = order_items.order_id
WHERE order_status='delivered' and
      order_purchase_timestamp BETWEEN date('{intial_date}') AND date('{final_date}')
GROUP BY EXTRACT(MONTH FROM order_purchase_timestamp)
'''
to_pandas(db.execute(sales_per_month_query), ['month','total_sales']).sort_values('month')